<a href="https://colab.research.google.com/github/machine-perception-robotics-group/MPRGDeepLearningLectureNotebook/blob/master/13_rnn/04_Seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#エンコーダ・デコーダ

リカレントニューラルネットワークは，系列データ内の関連性を内部状態として保持することができます．
この内部状態を利用して，新たな出力ができるようにした構造としてエンコーダ・デコーダがあります．
エンコーダ側に系列データを入力して，中間層では系列データ内の関連性を内部状態を形成します．
デコーダ側には内部状態を与えることで，内部状態を反映した何かしらの結果を出力します．
この応用が，google 翻訳などの機械翻訳です．

<img src="https://drive.google.com/uc?export=view&id=1zFl4Mjo4IRSQWSczJ4PzPkd53YJkb1oM" width = 100%>



##計算機の実装
ここでは，エンコーダ・デコーダ構造で計算機（足し算）を作ってみます．
このエンコーダ・デコーダ構造のことをSeq2seqと呼びます．

###データローダの作成
まず，データローダを用意します．データは0から9までの数字と加算記号，開始，終了のフラグです．また，３桁の数字の足し算を行うため，各桁の値を１つずつランダムに生成して連結しています．


In [ ]:
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

word2id = {str(i): i for i in range(10)}
word2id.update({"<pad>": 10, "+": 11, "<eos>": 12})
id2word = {v: k for k, v in word2id.items()}

class CalcDataset(torch.utils.data.Dataset):

    def transform(self, string, seq_len=7):
        tmp = []
        for i, c in enumerate(string):
            try:
                tmp.append(word2id[c])
            except:
                tmp += [word2id["<pad>"]] * (seq_len - i)
                break
        return tmp

    def __init__(self, data_num, train=True):
        super().__init__()
        self.data_num = data_num
        self.train = train
        self.data = []
        self.label = []

        for _ in range(data_num):
            x = int("".join([random.choice(list("0123456789")) for _ in range(random.randint(1, 3))] ))
            y = int("".join([random.choice(list("0123456789")) for _ in range(random.randint(1, 3))] ))
            left = ("{:*<7s}".format(str(x) + "+" + str(y))).replace("*", "<pad>")
            self.data.append(self.transform(left))

            z = x + y
            right = ("{:*<6s}".format(str(z))).replace("*", "<pad>")
            right = self.transform(right, seq_len=5)
            right = [12] + right
            right[right.index(10)] = 12
            self.label.append(right)
        


        self.data = np.asarray(self.data)
        self.label = np.asarray(self.label)

    def __getitem__(self, item):
        d = self.data[item]
        l = self.label[item]
        return d, l

    def __len__(self):
        return self.data.shape[0]

###エンコーダ・デコーダの作成
エンコーダとデコーダを用意します．エンコーダは，ワードエンベディングという特徴表現に変換する層とGRU層から構成されています．デコーダも同様の構造です．エンコーダ側の中間層の値がstateとして出力され，デコーダ側の中間層に入力されます．
エンコーダとデコーダは別々のネットワークとして用意し，それぞれの最適化にはAdamを利用します．

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


embedding_dim = 16
hidden_dim = 128
vocab_size = len(word2id)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class Encoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, batch_size=100):
        super(Encoder, self).__init__()
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=word2id["<pad>"])
        self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True)

    def forward(self, indices):
        embedding = self.word_embeddings(indices)
        if embedding.dim() == 2:
            embedding = torch.unsqueeze(embedding, 1)
        _, state = self.gru(embedding, torch.zeros(1, self.batch_size, self.hidden_dim, device=device))
        
        return state


class Decoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, batch_size=100):
        super(Decoder, self).__init__()
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=word2id["<pad>"])
        self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.output = nn.Linear(hidden_dim, vocab_size)

    def forward(self, index, state):
        embedding = self.word_embeddings(index)
        if embedding.dim() == 2:
            embedding = torch.unsqueeze(embedding, 1)
        gruout, state = self.gru(embedding, state)
        output = self.output(gruout)
        return output, state


encoder = Encoder(vocab_size, embedding_dim, hidden_dim, batch_size=100).to(device)
decoder = Decoder(vocab_size, embedding_dim, hidden_dim, batch_size=100).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=word2id["<pad>"])

# Initialize opotimizers
encoder_optimizer = optim.Adam(encoder.parameters(), lr=0.001)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=0.001)

###学習
学習を行います．学習データを2万サンプル生成して，データローダに与えます．
学習は200エポック行います．エンコーダの入力は数字または開始・終了・加算記号です．
デコーダの入力は計算結果です．
具体的には，54+37 を行う時，
エンコーダには，まず開始記号を最初に入力し，次に，5, 4, +, 3, 7 を入力します．そして，最後に終了記号を入力します．その時の中間層の情報をhidden_stateとしてエンコーダから受け取ります．
デコーダは，開始記号と中間情報(hidden_state)を最初に入力します，そして，計算結果の9, 1 を入力し，最後に終了記号を入力します．
この時，デコーダは各数字（または記号）の確率をdecoder_outputとして出力します．
decoder_outputは，[バッチサイズ, 1, 各クラス確率]の３次元なので，squeezeによって，[バッチサイズ,  各クラス確率] に次元削減します．
そして，クロスエントロピー誤差関数によって，ロスを求めます．
これを正解の長さ(=5)分繰り返し行い，ロスを累積します．
その後，誤差逆伝播，デコーダ，エンコーダの更新を行います．


In [ ]:
import numpy as np
from time import time

# GPUの確認
use_cuda = torch.cuda.is_available()
print('Use CUDA:', use_cuda)

batch_size=100
epoch_num = 200

train_data = CalcDataset(data_num = 20000)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

start = time()
for epoch in range(1, epoch_num+1):
    for data, label in train_loader:
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        if use_cuda:
            data = data.cuda()
            label = label.cuda()

        encoder_hidden = encoder(data)
        source = label[:, :-1]
        target = label[:, 1:]
        decoder_hidden = encoder_hidden

        loss = 0
        for i in range(source.size(1)):
            decoder_output, decoder_hidden = decoder(source[:, i], decoder_hidden)
            decoder_output = torch.squeeze(decoder_output)
            loss += criterion(decoder_output, target[:, i])

        # Perform backpropagation
        loss.backward()
        
        # Adjust model weights
        encoder_optimizer.step()
        decoder_optimizer.step()

  
    elapsed_time = time() - start
    if epoch % 10 == 0:
        print("epoch: {}, mean loss: {}, elapsed_time: {}".format(epoch, loss.item(), elapsed_time))
        
model_name = "seq2seq_calculator_v{}.pt".format(epoch)
torch.save({
    'encoder_model': encoder.state_dict(),
    'decoder_model': decoder.state_dict(),
}, model_name)


Use CUDA: True
epoch: 10, mean loss: 2.689300060272217, elapsed_time: 18.220115184783936
epoch: 20, mean loss: 0.9901413917541504, elapsed_time: 37.50547909736633
epoch: 30, mean loss: 0.46048635244369507, elapsed_time: 56.1470832824707
epoch: 40, mean loss: 0.2711130380630493, elapsed_time: 74.9522852897644
epoch: 50, mean loss: 0.31738555431365967, elapsed_time: 93.17892289161682
epoch: 60, mean loss: 0.0688471719622612, elapsed_time: 111.8789234161377
epoch: 70, mean loss: 0.36980101466178894, elapsed_time: 130.59371519088745
epoch: 80, mean loss: 0.359742134809494, elapsed_time: 148.91031122207642
epoch: 90, mean loss: 0.016640793532133102, elapsed_time: 167.2922763824463
epoch: 100, mean loss: 0.01231339294463396, elapsed_time: 185.7539083957672
epoch: 110, mean loss: 0.01959720067679882, elapsed_time: 204.26842951774597
epoch: 120, mean loss: 1.2317143678665161, elapsed_time: 222.58245587348938
epoch: 130, mean loss: 0.007878083735704422, elapsed_time: 241.07030415534973
epoch: 1

###評価
次に，学習したモデルを評価をします．テストデータを2000サンプル生成して，データローダに与えます．
ここで，学習時はエンコーダとデコーダのバッチサイズを100としていました．
テスト時は１つずつ行いたいので，エンコーダとデコーダを新たに生成し，学習したパラメータをロードします．
エンコーダ側に計算したい数字（または記号）を入力して中間情報stateを得ます．
デコーダ側に，中間情報stateと開始記号<eos>を入力します．
デコーダ側の出力は数字または記号(token)と中間情報です．
これらを繰り返しデコーダに入力します．<eos>が出力されたら繰り返しは終了です．
出力されたtokenを追加したリストrightを計算結果とします．
計算する式(left)を作成した後，evalでその計算結果が正しいかどうかを判定します．



In [ ]:

batch_size = 1
test_data = CalcDataset(data_num = 2000)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False)

encoder = Encoder(vocab_size, embedding_dim, hidden_dim, batch_size=1).to(device)
decoder = Decoder(vocab_size, embedding_dim, hidden_dim, batch_size=1).to(device)

model_name = "seq2seq_calculator_v{}.pt".format(epoch)
checkpoint = torch.load(model_name)
encoder.load_state_dict(checkpoint["encoder_model"])
decoder.load_state_dict(checkpoint["decoder_model"])

accuracy = 0
        
# 評価の実行   
with torch.no_grad():
    for data, label in test_loader:
        if use_cuda:
            data = data.cuda()

        state = encoder(data)

        right = []
        token = "<eos>"
        for _ in range(7):
            index = word2id[token]
            input_tensor = torch.tensor([index], device=device)
            output, state = decoder(input_tensor, state)
            prob = F.softmax(torch.squeeze(output))
            index = torch.argmax(prob.cpu().detach()).item()
            token = id2word[index]
            if token == "<eos>":
                break
            right.append(token)
        right = "".join(right)
        
        x = list(data[0].to('cpu').detach().numpy() )
        try:
            padded_idx_x = x.index(word2id["<pad>"])
        except ValueError:
            padded_idx_x = len(x)
        left = "".join(map(lambda c: str(id2word[c]), x[:padded_idx_x]))



        flag = ["F", "T"][eval(left) == int(right)]
        print("{:>7s} = {:>4s} :{}".format(left, right, flag))
        if flag == "T":
            accuracy += 1
print("Accuracy: {:.2f}".format(accuracy / len(test_loader)))



   8+12 =   20 :T
   89+5 =   94 :T
   8+23 =   31 :T
    3+6 =    9 :T
  0+276 =  276 :T
   6+47 =   54 :F
   80+6 =   86 :T
 657+43 =  700 :T
    6+9 =   15 :T
  79+99 = 1069 :F
   1+58 =   60 :F
   74+6 =   81 :F
   3+12 =   15 :T
393+152 =  634 :F
  44+64 =  108 :T
   6+78 =   84 :T
  19+52 =   71 :T
  757+2 =  769 :F
  7+791 =  798 :T
 316+68 =  364 :F
   64+8 =   72 :T
  890+3 =  907 :F
   37+2 =   39 :T
854+918 = 1764 :F
  32+77 =  109 :T
  6+768 =  773 :F
  487+5 =  492 :T
   5+87 =   92 :T
   41+9 =   50 :T


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


  77+75 =  152 :T
  9+212 =  211 :F
 646+88 =  745 :F
  87+81 =  168 :T
  4+697 =  701 :T
 622+90 =  732 :F
    3+3 =    6 :T
  84+94 =  178 :T
   0+48 =   48 :T
  818+2 =  820 :T
 42+775 =  818 :F
796+336 = 1261 :F
 235+66 =  301 :T
   3+47 =   50 :T
976+249 = 1244 :F
  7+824 =  831 :T
    7+4 =   11 :T
 700+23 =  733 :F
 33+706 =  750 :F
  6+749 =  754 :F
  1+639 =  630 :F
806+172 = 1069 :F
510+446 =  926 :F
 170+90 =  250 :F
  269+5 =  274 :T
   6+62 =   68 :T
 41+579 =  619 :F
  4+576 =  580 :T
    3+4 =    7 :T
  558+0 =  559 :F
    7+2 =    9 :T
    0+7 =    7 :T
   2+82 =   84 :T
 946+55 = 1021 :F
  4+719 =  733 :F
  22+47 =   79 :F
  4+793 =  787 :F
  5+770 =  775 :T
 350+44 =  404 :F
   29+7 =   55 :F
    8+1 =    9 :T
    5+6 =   11 :T
  415+9 =  424 :T
   64+8 =   72 :T
 502+42 =  553 :F
   2+10 =   12 :T
   4+44 =   48 :T
649+507 = 1155 :F
 873+86 =  959 :T
102+595 =  778 :F
   39+7 =   46 :T
   75+0 =   75 :T
   3+96 =   99 :T
    8+6 =   14 :T
  5+558 =  563 :T
   18+1 = 

#課題
* 足し算だけでなく，色々な四則演算を試そう
* 他のリカレントニューラルネットワークを使って精度比較をしてみよう